In [2]:
import pandas as pd
from plotly import express as px

# Exploratory data analysis

In [3]:
df = pd.read_csv('dashboard/data/commodity_trade_statistics_data.csv', low_memory=False)
df.sample(5)

,country_or_area,year,comm_code,commodity,flow,trade_usd,weight_kg,quantity_name,quantity,category
5782203,Côte d'Ivoire,2003,680293,Worked granite,Export,14470,3907.0,Weight in kilograms,3907.0,68_stone_plaster_cement_asbestos_mica_etc_arti...
2752757,Croatia,2004,320415,Vat dyes and preparations based thereon,Export,4112,225.0,Weight in kilograms,225.0,32_tanning_dyeing_extracts_tannins_derivs_pigm...
763630,Kyrgyzstan,2000,090210,"Tea, green (unfermented) in packages < 3 kg",Export,19119,16101.0,Weight in kilograms,16101.0,09_coffee_tea_mate_and_spices
3179379,Italy,2009,370510,"Photo plates and film, exposed & developed, fo...",Export,3389860,51090.0,Weight in kilograms,51090.0,37_photographic_or_cinematographic_goods
8074568,Antigua and Barbuda,2010,960899,"Duplicating stylos, pen/pencil holders, pen parts",Import,954,279.0,Weight in kilograms,279.0,96_miscellaneous_manufactured_articles


In [4]:
print(f'{len(df.index)} rows')
df.nunique()

8225871 rows


country_or_area        209
year                    29
comm_code             5040
commodity             5031
flow                     4
trade_usd          3062165
weight_kg          2137907
quantity_name           12
quantity           2124833
category                98
dtype: int64

In [5]:
# Quantitative values
df.describe().T

,count,mean,std,min,25%,50%,75%,max
year,8225871.0,2.004768e+03,7.037779e+00,1988.0,1999.0,2005.0,2011.00,2.016000e+03
trade_usd,8225871.0,9.505370e+07,7.074154e+09,1.0,11072.0,153416.0,1895476.50,2.443311e+12
weight_kg,8097396.0,3.556503e+07,2.479277e+09,0.0,1089.0,26898.0,447443.25,1.860133e+12
quantity,7921014.0,2.671470e+08,3.739231e+11,0.0,1062.0,27120.0,462362.00,1.026357e+15


In [6]:
# Categorical values
df.describe(include='O')

,country_or_area,comm_code,commodity,flow,quantity_name,category
count,8225871,8225871,8225871,8225871,8225871,8225871
unique,209,5040,5031,4,12,98
top,Australia,TOTAL,ALL COMMODITIES,Import,Weight in kilograms,95_toys_games_sports_requisites
freq,307627,9185,9185,4848524,6617980,100000


### Checking for missing values

In [7]:
# Missing values
cols_with_nulls = df.columns[df.isnull().any()].tolist()
print(df[cols_with_nulls].isnull().sum())

weight_kg    128475
quantity     304857
dtype: int64


### **Substantial amount of countries do not to seem to export or import oil.**  Likely the dataset has gaps in data. Analyses regarding the trade-value or the overall import and export will be skewed.

### Possible outlier between years 1995-1997, sudden jump in quantity traded.

In [8]:
df_wq=df.groupby(['year'],as_index=True)[['weight_kg','quantity','trade_usd']].agg('sum')
fig = px.line(df_wq, x=df_wq.index, y=df_wq.columns, title='Quantity, Weight and Price of Commodities Traded Globally')
fig.show()

# Value of goods and services

### Plot export of goods (Chart 4)

In [24]:
# Value of exported of goods and services
df_export = df[ (df['flow'] == 'Export') | (df['flow'] == 'Re-Export')]
# Select top 10 countries by value of export
top_10_countries = df_export.groupby(['country_or_area'])[['trade_usd']].sum().sort_values(by=['trade_usd'], ascending=False)[:10].index
df_export = df_export[df_export['country_or_area'].isin(top_10_countries)]

df_export = df_export.groupby(['year','country_or_area'])[['weight_kg', 'quantity','trade_usd']].sum()
df_export.sort_values(by=['year'], inplace=True)
df_export.reset_index(inplace=True)
# value = 'trade_usd'
label = 'Value of Export in USD'
export_chart = px.line(df_export, x='year', y='trade_usd', title='Export of Goods and Services', color='country_or_area',
                       labels={'trade_usd': label}, color_discrete_sequence=px.colors.qualitative.Dark24,)
#color_discrete_sequence=px.colors.qualitative.Dark24,
export_chart.show()
# TODO allow selection of wordl map, line chart or a table